## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chicama,PE,-7.8447,-79.1469,59.54,65,76,6.31,Clouds
1,1,Bredasdorp,ZA,-34.5322,20.0403,74.28,60,28,9.22,Clouds
2,2,Marrakesh,MA,31.6315,-8.0083,84.27,24,0,6.91,Clear
3,3,Busselton,AU,-33.6500,115.3333,50.04,90,57,7.76,Clouds
4,4,Urengoy,RU,65.9667,78.3667,13.89,95,99,10.78,Clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chicama,PE,-7.8447,-79.1469,59.54,65,76,6.31,Clouds
1,1,Bredasdorp,ZA,-34.5322,20.0403,74.28,60,28,9.22,Clouds
3,3,Busselton,AU,-33.6500,115.3333,50.04,90,57,7.76,Clouds
6,6,Rikitea,PF,-23.1203,-134.9692,74.84,83,78,21.23,Clouds
10,10,Bethel,US,41.3712,-73.4140,61.50,96,90,4.00,Rain
11,11,Vaini,TO,-21.2000,-175.2000,71.76,94,20,8.05,Clouds
12,12,Pisco,PE,-13.7000,-76.2167,65.01,88,90,4.61,Mist
14,14,Bambous Virieux,MU,-20.3428,57.7575,75.45,73,40,5.75,Clouds
15,15,Biak,ID,-0.9131,122.8766,76.60,86,71,2.73,Clouds
16,16,Santa Ana,US,33.7456,-117.8678,67.77,89,90,3.44,Haze


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                340
City                   340
Country                338
Lat                    340
Lng                    340
Max Temp               340
Humidity               340
Cloudiness             340
Wind Speed             340
Current Description    340
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chicama,PE,59.54,Clouds,-7.8447,-79.1469,
1,Bredasdorp,ZA,74.28,Clouds,-34.5322,20.0403,
3,Busselton,AU,50.04,Clouds,-33.6500,115.3333,
6,Rikitea,PF,74.84,Clouds,-23.1203,-134.9692,
10,Bethel,US,61.50,Rain,41.3712,-73.4140,
11,Vaini,TO,71.76,Clouds,-21.2000,-175.2000,
12,Pisco,PE,65.01,Mist,-13.7000,-76.2167,
14,Bambous Virieux,MU,75.45,Clouds,-20.3428,57.7575,
15,Biak,ID,76.60,Clouds,-0.9131,122.8766,
16,Santa Ana,US,67.77,Haze,33.7456,-117.8678,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Chicama,PE,59.54,Clouds,-7.8447,-79.1469,Balihai Bungalows
1,Bredasdorp,ZA,74.28,Clouds,-34.5322,20.0403,Bredasdorp Country Manor
3,Busselton,AU,50.04,Clouds,-33.6500,115.3333,Observatory Guest House
6,Rikitea,PF,74.84,Clouds,-23.1203,-134.9692,People ThankYou
10,Bethel,US,61.50,Rain,41.3712,-73.4140,Courtyard by Marriott Danbury
11,Vaini,TO,71.76,Clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Pisco,PE,65.01,Mist,-13.7000,-76.2167,Hotel San Isidro Oficial
14,Bambous Virieux,MU,75.45,Clouds,-20.3428,57.7575,Casa Tia Villa
15,Biak,ID,76.60,Clouds,-0.9131,122.8766,Penginapan Kim
16,Santa Ana,US,67.77,Haze,33.7456,-117.8678,Santa Ana Travel Inn


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))